## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Clinical De-identification for Italian](https://aws.amazon.com/marketplace/pp/prodview-p3syjgme2b6w4).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Italian

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: [it.deid.clinical](https://nlp.johnsnowlabs.com/2023/06/17/clinical_deidentification_it.html)
- **Model Description**: This pipeline can be used to deidentify PHI information from medical texts in Italian. The pipeline can mask and obfuscate the following entities: DATE, AGE, SEX, PROFESSION, ORGANIZATION, PHONE, E-MAIL, ZIP, STREET, CITY, COUNTRY, PATIENT, DOCTOR, HOSPITAL, MEDICALRECORD, SSN, IDNUM, ACCOUNT, PLATE, USERNAME, URL, and IPADDR.

In [24]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [25]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [26]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [27]:
model_name = "it-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [28]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

### Important Parameter

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**PZ: Giancarlo Binaghi, CODICE FISCALE: MVANSK92F09W408A, INDIRIZZO: Viale Burcardo 7, CITTÀ : Napoli**"

    - **masked**: Default policy that masks entities with their type.

      -> 'PZ: `<DOCTOR>`, CODICE FISCALE: `<SSN>`, INDIRIZZO: `<STREET>`, CITTÀ : `<CITY>`'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'PZ:`Germana Maglio-Dovara`, CODICE FISCALE: `ECI-QLN77G15L455Y`, INDIRIZZO: `Viale Orlando 808`, CITTÀ : `Sesto Raimondo`'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'PZ: `****`, CODICE FISCALE: `****`, INDIRIZZO: `****`, CITTÀ : `****`'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'PZ: [`***************`], CODICE FISCALE: [`**************`], INDIRIZZO: [`**************`], CITTÀ : [`****`]'

    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [30]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [31]:
docs = [
'''PZ: Giancarlo Binaghi, CODICE FISCALE: MVANSK92F09W408A, INDIRIZZO: Viale Burcardo 7, CITTÀ : Napoli... 
PZ: Giancarlo Binaghi, CODICE FISCALE: MVANSK92F09W408A, INDIRIZZO: Viale Burcardo 7, CITTÀ : Napoli
CODICE POSTALE: 80139,  DATA DI NASCITA: 03/03/1946, ETÀ: 70 anni , SESSO: M, EMAIL: gcbinaghi@tim.it, DATA DI AMMISSIONE: 12/12/2016, DOTTORE: Eva Viviani, RAPPORTO CLINICO: 70 anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.
È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.
L'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; l'esame digitale rettale era coerente con un adenoma prostatico di grado I/IV.
L'analisi delle urine ha mostrato 4 globuli rossi/campo e 0-5 leucociti/campo; il resto del sedimento era normale.
L'emocromo è normale; la biochimica ha mostrato una glicemia di 169 mg/dl e trigliceridi 456 mg/dl; la funzione epatica e renale sono normali. PSA di 1,16 ng/ml. INDIRIZZATO A: Dott. Bruno Ferrabosco - ASL Napoli 1 Centro, Dipartimento di Endocrinologia e Nutrizione - Stretto Scamarcio 320, 80138 Napoli, EMAIL: bferrabosco@poste.it''',

'''PZ: Giustino Adinolfi, CODICE FISCALE: DHQFOH39E45C604H, INDIRIZZO: Canale Seddio 62, CITTÀ: Palermo... 
PZ: Giustino Adinolfi, CODICE FISCALE: DHQFOH39E45C604H, INDIRIZZO: Canale Seddio 62, CITTÀ: Palermo, CODICE POSTALE: 90128,  DATA DI NASCITA: 15/07/1979,  ETÀ: 38 anni , SESSO: M, DATA DI AMMISSIONE: 04/03/2014, DOTTORE: Natalia Tamborini
RAPPORTO CLINICO: Storia attuale: paziente maschio di 38 anni, valutato in consultazione per una coxalgia destra di un anno e mezzo di evoluzione, di caratteristiche meccaniche. Ha riportato dolore con l'esercizio fisico e una leggera limitazione della mobilità in certe posture. Meccanico di professione, nel suo tempo libero andava in bicicletta (40 km al giorno nei fine settimana), correva 3 volte alla settimana per 50 minuti e camminava per un'ora nei giorni in cui non correva. Il dolore è aumentato progressivamente e attualmente cammina soltanto.
ESAME FISICO: L'esame fisico ha rivelato un dolore alla palpazione nel terzo medio della regione inguinale destra con dolore alla flessione, all'adduzione e alla rotazione, specialmente alla rotazione interna (manovra di shock positivo). L'equilibrio dell'articolazione dell'anca è di 110o di flessione bilaterale, rotazione interna di 10o destra, 40o sinistra e rotazione esterna di 30o destra, 40o sinistra. L'equilibrio muscolare sulla scala Daniels è 5/5 globale bilaterale. Non c'è dolore alla palpazione dei punti sciatici e le manovre Valleix, Fabere, Lasegue e Bragard sono negative. Non c'era nemmeno dolore alla palpazione della banda iliotibiale e il test Ober era negativo. Nell'esame dell'andatura, abbiamo osservato un'assenza di claudicazione e l'appoggio monopodalico alternato destra-sinistra è stabile, con Trendelenburg negativo. Il paziente aveva dolore quando camminava con una flessione forzata dell'anca e del ginocchio.
Esami complementari: Una semplice radiologia anteroposteriore (AP) ha mostrato una deformità a forma di gobba nella regione femorale cervico-cefalica destra. L'emocromo richiesto, la biochimica, gli ormoni tiroidei, gli anticorpi e i reagenti di fase acuta erano normali.
Alla luce di questi risultati, è stata richiesta una risonanza magnetica dell'anca, che ha mostrato una leggera alterazione subcondrale, con edema osseo nell'anca destra, con irregolarità corticale e leggera sinovite nell'interlinea articolare, con distensione della borsa ilio-psoas. La testa femorale ha un piccolo isolotto osseo e l'angolo alfa è maggiore di 50o. L'artrografia mostra un pizzicamento dell'interlinea articolare antero-superiore e postero-inferiore, un colletto osteofita femorale incipiente con piccole lesioni subcondrali antero-superiore della testa femorale e piccoli focolai di edema nell'acetabolo anterosuperiore. Non c'è necrosi avascolare o lesioni nel labrum. Una lesione cistica juxta-articolare è anche visualizzata in relazione al grande trocantere.
Evoluzione: Con la diagnosi di sindrome da impingement femoroacetabolare di tipo LEVA (nota anche come CAM), al paziente è stato prescritto un trattamento di magnetoterapia (15 sedute) per migliorare i sintomi del dolore e il trofismo osseo.
Il paziente è stato indirizzato all'unità di artroscopia del servizio di traumatologia per la valutazione della chirurgia artroscopica.
Diagnosi principale: sindrome da impingement femoroacetabolare di tipo LEVA.
TRATTAMENTO: Il trattamento farmacologico è prescritto con un'associazione di glucosamina-condroitina solfato orale 400 mg ogni 12 ore per 3 mesi e farmaci antinfiammatori non steroidei al bisogno se il dolore è presente. Viene istruita a riposare negli sport per evitare lo shock all'articolazione, con un'attività fisica attiva controllata, raccomandando di non superare i 90 gradi di flessione dell'anca ed evitando le rotazioni. Prenda un appuntamento per un controllo una volta terminati i solfati di glucosamina-condroitina per vedere il miglioramento clinico. Chiami il numero di assistenza e appuntamenti precedenti dell'ospedale 914 90 66 66 60.
RIFERITO DA: Dott. Paloma Trupiano. Azienda Ospedaliera San Giovanni Addolorata, Strada Damiano 68, 90128 Palermo. Telefono: +39 3664066420. E-mail: p_trupiano@alice.it ''',

]


sample_text = """PZ: Lina Quasimodo
CODICE FISCALE: DHQFOH39E45C604H
INDIRIZZO: Viale Bianca 95
CITTÀ: Medesano
CODIC... 
PZ: Lina Quasimodo
CODICE FISCALE: DHQFOH39E45C604H
INDIRIZZO: Viale Bianca 95
CITTÀ: Medesano
CODICE POSTALE: 43014 
DATA DI NASCITA: 05/10/1976
ETÀ: 44 anni 
SESSO: Femminile
DATA DI AMMISSIONE: 04/03/2014
DOTTORE: Lina Antonucci
RAPPORTO CLINICO: Donna di 44 anni, si è rivolta alla nostra unità a causa di un aumento di volume nella regione zigomatica e della progressiva limitazione dell'apertura della bocca di 10 mesi di evoluzione. L'anamnesi medica non era rilevante. 
L'esame fisico extraorale ha mostrato un'asimmetria facciale dovuta a un aumento di volume nella guancia sinistra, di limiti diffusi, di consistenza ossea, indolore, con pelle di aspetto normale. L'apertura della bocca era di 30 mm. L'articolazione temporomandibolare (ATM) era normale alla palpazione, senza rumori o dolore all'apertura. All'ortopantomografia abbiamo osservato un processo coronoideo sinistro allargato. L'ipotesi diagnostica era un tumore coronoide.
Prove complementari ed evoluzione: Un'incisione è stata fatta sul bordo anteriore del ramus mandibolare, disinserendo tutte le inserzioni del muscolo temporale. Una pseudocapsula fibrosa è stata trovata intorno all'allargamento, che è stata rilasciata. È stata eseguita una coronoidectomia. L'apertura buccale è stata immediatamente ripristinata a 43 mm. Il decorso postoperatorio è stato poco movimentato e il paziente è stato dimesso dopo 48 ore.
Lo studio istopatologico ha riportato la presenza di fibre, neoformazione ossea e tessuto cartilagineo ialino. La diagnosi era osteocondroma. I controlli successivi sono stati senza problemi. La dimissione definitiva è stata data a 10 mesi, con un rimodellamento quasi completo della deformità della guancia.

RIFERITO DA: Dott. Annunziata Luna. AZIENDA OSPEDALIERA G. BROTZU, Vicolo Margherita 6, 85057 Tramutola. Telefono: 018718869. E-mail: luna@libero.it """

### JSON

#### Example 1: masked (default-policy)

In [32]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

PZ: <PATIENT>
CODICE FISCALE: <SSN>
INDIRIZZO: <STREET>
CITTÀ: <CITY>
CODIC... 
PZ: <PATIENT>
CODICE FISCALE: <SSN>
INDIRIZZO: <STREET>
CITTÀ: <CITY>
CODICE POSTALE: <ZIP> 
DATA DI NASCITA: <DATE>
ETÀ: <AGE>anni 
SESSO: <SEX>
DATA DI AMMISSIONE: <DATE>
DOTTORE: <DOCTOR>
RAPPORTO CLINICO: <SEX> di <AGE>anni, si è rivolta alla nostra unità a causa di un aumento di volume nella regione zigomatica e della progressiva limitazione dell'apertura della bocca di 10 mesi di evoluzione. L'anamnesi medica non era rilevante. 
L'esame fisico extraorale ha mostrato un'asimmetria facciale dovuta a un aumento di volume nella guancia sinistra, di limiti diffusi, di consistenza ossea, indolore, con pelle di aspetto normale. L'apertura della bocca era di 30 mm. L'articolazione temporomandibolare (ATM) era normale alla palpazione, senza rumori o dolore all'apertura. All'ortopantomografia abbiamo osservato un processo coronoideo sinistro allargato. L'ipotesi diagnostica era un tumore coronoide.
Prove comple

#### Example 2: obfuscated

In [34]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

PZ: Lollobrigida
CODICE FISCALE: YILQTI62H78P937I
INDIRIZZO: Rotonda Victoria 1 Appartamento 12
CITTÀ: Borgo Pomponio
CODIC... 
PZ: Lollobrigida
CODICE FISCALE: YILQTI62H78P937I
INDIRIZZO: Rotonda Victoria 1 Appartamento 12
CITTÀ: Borgo Pomponio
CODICE POSTALE: 76347 
DATA DI NASCITA: 05/10/1976
ETÀ: 47anni 
SESSO: La Pz
DATA DI AMMISSIONE: 04/03/2014
DOTTORE: Sig.ra Tina Zichichi
RAPPORTO CLINICO: maschio di 47anni, si è rivolta alla nostra unità a causa di un aumento di volume nella regione zigomatica e della progressiva limitazione dell'apertura della bocca di 10 mesi di evoluzione. L'anamnesi medica non era rilevante. 
L'esame fisico extraorale ha mostrato un'asimmetria facciale dovuta a un aumento di volume nella guancia sinistra, di limiti diffusi, di consistenza ossea, indolore, con pelle di aspetto normale. L'apertura della bocca era di 30 mm. L'articolazione temporomandibolare (ATM) era normale alla palpazione, senza rumori o dolore all'apertura. All'ortopantomografia abbiamo 

### JSON Lines

In [36]:
def create_jsonl(records, masking_policy=None):
    json_records = []

    if isinstance(records, str):
        records = [records]

    for text in records:
        record = {"text": text}

        if masking_policy is not None:
            record["masking_policy"] = masking_policy
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record, ensure_ascii=False) for record in json_records)
    return json_lines


#### Example 1: masked (default-policy)

In [37]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="masked")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "PZ: <PATIENT>\nCODICE FISCALE: <SSN>\nINDIRIZZO: <STREET>\nCITTÀ: <CITY>\nCODIC... \nPZ: <PATIENT>\nCODICE FISCALE: <SSN>\nINDIRIZZO: <STREET>\nCITTÀ: <CITY>\nCODICE POSTALE: <ZIP> \nDATA DI NASCITA: <DATE>\nETÀ: <AGE>anni \nSESSO: <SEX>\nDATA DI AMMISSIONE: <DATE>\nDOTTORE: <DOCTOR>\nRAPPORTO CLINICO: <SEX> di <AGE>anni, si è rivolta alla nostra unità a causa di un aumento di volume nella regione zigomatica e della progressiva limitazione dell'apertura della bocca di 10 mesi di evoluzione. L'anamnesi medica non era rilevante. \nL'esame fisico extraorale ha mostrato un'asimmetria facciale dovuta a un aumento di volume nella guancia sinistra, di limiti diffusi, di consistenza ossea, indolore, con pelle di aspetto normale. L'apertura della bocca era di 30 mm. L'articolazione temporomandibolare (ATM) era normale alla palpazione, senza rumori o dolore all'apertura. All'ortopantomografia abbiamo osservato un processo coronoideo sinistro allargato. L'ipotesi diagnostica era 

#### Example 2: obfuscated

In [39]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="obfuscated")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "PZ: Lollobrigida\nCODICE FISCALE: YILQTI62H78P937I\nINDIRIZZO: Rotonda Victoria 1 Appartamento 12\nCITTÀ: Borgo Pomponio\nCODIC... \nPZ: Lollobrigida\nCODICE FISCALE: YILQTI62H78P937I\nINDIRIZZO: Rotonda Victoria 1 Appartamento 12\nCITTÀ: Borgo Pomponio\nCODICE POSTALE: 76347 \nDATA DI NASCITA: 05/10/1976\nETÀ: 47anni \nSESSO: La Pz\nDATA DI AMMISSIONE: 04/03/2014\nDOTTORE: Sig.ra Tina Zichichi\nRAPPORTO CLINICO: maschio di 47anni, si è rivolta alla nostra unità a causa di un aumento di volume nella regione zigomatica e della progressiva limitazione dell'apertura della bocca di 10 mesi di evoluzione. L'anamnesi medica non era rilevante. \nL'esame fisico extraorale ha mostrato un'asimmetria facciale dovuta a un aumento di volume nella guancia sinistra, di limiti diffusi, di consistenza ossea, indolore, con pelle di aspetto normale. L'apertura della bocca era di 30 mm. L'articolazione temporomandibolare (ATM) era normale alla palpazione, senza rumori o dolore all'apertur

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [42]:
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [43]:
# Create JSON and JSON Lines data
input_json_data = {
    "input1.json": json.dumps({"text": docs, "masking_policy": "masked"}, ensure_ascii=False),
    "input2.json": json.dumps({"text": docs, "masking_policy": "obfuscated"}, ensure_ascii=False),
    "input3.json": json.dumps({"text": docs, "masking_policy": "masked_fixed_length_chars"}, ensure_ascii=False),
    "input4.json": json.dumps({"text": docs, "masking_policy": "masked_with_chars"}, ensure_ascii=False),
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}

# Upload JSON and JSON Lines data to S3
for file_name, data in input_json_data.items():
    upload_to_s3(data, file_name)

for file_name, data in input_jsonl_data.items():
    upload_to_s3(data, file_name)


### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [45]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [46]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_json_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------


{'predictions': ["PZ: <DOCTOR>, CODICE FISCALE: <SSN>, INDIRIZZO: <STREET>, CITTÀ : <COUNTRY>... \nPZ: <DOCTOR>, CODICE FISCALE: <SSN>, INDIRIZZO: <STREET>, CITTÀ : <PATIENT>\nCODICE POSTALE: <ZIP>  DATA DI NASCITA: <DATE>, ETÀ: <AGE>anni , SESSO: <SEX>, EMAIL: <E-MAIL>, DATA DI AMMISSIONE: <DATE>, DOTTORE: <DOCTOR>, RAPPORTO CLINICO: <AGE>anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.\nÈ stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.\nL'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; l'esame digitale rettale era coerente con un adenoma prosta



-------------------------------------------------- obfuscated --------------------------------------------------


{'predictions': ["PZ: Giulia, CODICE FISCALE: BEXKDJ25Q32N731X, INDIRIZZO: Canale Adamo 11 Piano 7, CITTÀ : Palestina... \nPZ: Giulia, CODICE FISCALE: BEXKDJ25Q32N731X, INDIRIZZO: Canale Adamo 11 Piano 7, CITTÀ : Palestina\nCODICE POSTALE: 13462  DATA DI NASCITA: 03/03/1946, ETÀ: 79anni , SESSO: U, EMAIL: HenryWatson@world.com, DATA DI AMMISSIONE: 12/12/2016, DOTTORE: Schiavone, RAPPORTO CLINICO: 79anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.\nÈ stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.\nL'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali;



-------------------------------------------------- masked_fixed_length_chars --------------------------------------------------


{'predictions': ["PZ: ****, CODICE FISCALE: ****, INDIRIZZO: ****, CITTÀ : ****... \nPZ: ****, CODICE FISCALE: ****, INDIRIZZO: ****, CITTÀ : ****\nCODICE POSTALE: ****  DATA DI NASCITA: ****, ETÀ: ****anni , SESSO: ****, EMAIL: ****, DATA DI AMMISSIONE: ****, DOTTORE: ****, RAPPORTO CLINICO: ****anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.\nÈ stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.\nL'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; l'esame digitale rettale era coerente con un adenoma prostatico di grado I/IV.\nL'analisi delle urine h



-------------------------------------------------- masked_with_chars --------------------------------------------------


{'predictions': ["PZ: [***************], CODICE FISCALE: [**************], INDIRIZZO: [**************], CITTÀ : [****]... \nPZ: [***************], CODICE FISCALE: [**************], INDIRIZZO: [**************], CITTÀ : [****]\nCODICE POSTALE: [***]  DATA DI NASCITA: [********], ETÀ: **anni , SESSO: *, EMAIL: [**************], DATA DI AMMISSIONE: [********], DOTTORE: [*********], RAPPORTO CLINICO: **anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.\nÈ stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.\nL'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; 

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [48]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [49]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_jsonlines_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------
{"predictions": "PZ: <DOCTOR>, CODICE FISCALE: <SSN>, INDIRIZZO: <STREET>, CITTÀ : <COUNTRY>... \nPZ: <DOCTOR>, CODICE FISCALE: <SSN>, INDIRIZZO: <STREET>, CITTÀ : <PATIENT>\nCODICE POSTALE: <ZIP>  DATA DI NASCITA: <DATE>, ETÀ: <AGE>anni , SESSO: <SEX>, EMAIL: <E-MAIL>, DATA DI AMMISSIONE: <DATE>, DOTTORE: <DOCTOR>, RAPPORTO CLINICO: <AGE>anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.\nÈ stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.\nL'esame fisico ha mostrato buone con

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

